In [ ]:
import os
import json
from tqdm import tqdm
from tensorflow import keras
from IPython.display import clear_output
from collections import Counter
import time

import matplotlib.pyplot as plt
import numpy as np


from pathlib import Path

In [ ]:
# Path(os.getcwd()+"\output\cnn_models").mkdir(parents=True, exist_ok=True)

# Load input data

* data: Tabular data + ground truth
* dict_col_candidate_classes: a dictionary with filename_columns and in each of the an array of [(candidate_type, candidate_entity, original_cell_value, rank)]
* type_neighours_pos_neg_samples: a dictionary that is used to train the classifiers so for each candidate class we have the neighbouring classes, positive samples from the KG and positive and negative samples from the tabular data

# only consider candidate classes between columns that have links

In [3]:
# Load the dictionary with the lookup results for each cell value in the tabular data
def load_json(data_json):
    with open(data_json) as json_file:
        return json.load(json_file)

In [4]:
output_folder = "output\\"
cnn_model_directory = os.getcwd()+'\\output\\cnn_models'


data = load_json(output_folder+'data.json')

dict_col_candidate_classes = load_json(output_folder+'dict_col_candidate_classes.json')


try:
    type_neighours_pos_neg_samples = load_json(output_folder+'type_neighours_pos_neg_samples.json')
except:
    pass

try:
    dict_cand = load_json(output_folder+'dict_cand.json')
except:
    pass

# Assess lookup efficiency

In this step, provided that we have the ground truth, we asses if the expected class is in the top x of the retrieved candidate classes.

In [5]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON, N3
from pprint import pprint
import requests
import xml.etree.ElementTree as ET

In [6]:
def dbo_sparql_results(query_string):
    
    classes = list([])
    dbo_prefix = 'http://dbpedia.org/ontology/'
    
    
    sparql = SPARQLWrapper('https://dbpedia.org/sparql')
    sparql.setQuery(query_string)
    
    try:
        sparql.setReturnFormat(JSON)
        qres = sparql.query().convert()
        for entity_class in qres['results']['bindings']:
            if dbo_prefix in entity_class[list(qres['results']['bindings'][0].keys())[0]]['value']:
                candicate_class = entity_class[list(qres['results']['bindings'][0].keys())[0]]['value'].split(dbo_prefix)[1]
                classes.append(candicate_class)
    except:
        pass
    
    return classes

def get_dbo_subclass(superClass):
    
    query_string = f'''
    SELECT distinct ?subClass 
    WHERE {{ ?subClass rdfs:subClassOf dbo:{superClass}. }}
    '''
    return dbo_sparql_results(query_string)


def get_dbo_superclass(subclass):
    
    query_string = f'''
    SELECT distinct ?superClass 
    WHERE {{ dbo:{subclass} rdfs:subClassOf ?superClass . }}
    '''
    
    return dbo_sparql_results(query_string)


def get_dbo_superclasses(subclass):
    classes = list([])
    
    try:
        parent = get_dbo_superclass(subclass)
    except:
        return []
    
    while len(parent) > 0:
        classes.append(parent[0])
        parent = get_dbo_superclass(parent[0])
    return classes

In [7]:
get_dbo_superclass('Film')

['Work']

In [8]:
def get_column_candidate_classes(dict_col_candidate_classes,filename,col,limit = -1):
    if limit >= 0:
        candidate_list = Counter([i[0] for i in dict_col_candidate_classes[filename][col]]).most_common()[:limit]
    else:
        candidate_list = Counter([i[0] for i in dict_col_candidate_classes[filename][col]]).most_common()
#     return [x[0] for x in candidate_list]
    return candidate_list

dict_cand = dict()

total = 0
for filename in dict_col_candidate_classes:
    dict_cand[filename] = dict()
    for col in dict_col_candidate_classes[filename]:
        dict_cand[filename][col] = dict()
        dict_cand[filename][col]['class_without_hr'] = get_column_candidate_classes(dict_col_candidate_classes,filename, col)
        total+=1

In [9]:
# dict_cand

On top of having a list of all candidate classes that appear in a column, the next step is to remove classes that are higher up in the list if at least one of their subclasses also appears in the list.
This approach seems to improve the precision (i.e. the top classes are more relevant) but decrease the recall

In [10]:
import pandas as pd
df_entities = pd.DataFrame()

rank_threshold = 5

for filename in dict_col_candidate_classes:
    for col in dict_col_candidate_classes[filename]:
        df_entities = df_entities.append(pd.DataFrame([[filename,col,x[0], x[1], x[2], x[3]] for x in dict_col_candidate_classes[filename][col]], columns=['filename', 'col','type', 'entity', 'cell_value', 'rank']))
        
df_entities = df_entities[df_entities['rank'] <= rank_threshold]
temp = df_entities.groupby(by=["filename", "col", "type"]).count()["entity"].reset_index().sort_values(by=['filename', 'col', 'entity'],ascending=False)

dict_candidate = dict()
for filename in list(temp.filename.unique()):
    dict_candidate[filename] = dict()
    for col in list(temp[temp.filename == filename].col.unique()):
        total = temp[(temp.filename==filename) & (temp.col == col)].sum().entity
        df = temp[(temp.filename==filename) & (temp.col == col)].copy()
        df['vote_pct'] = df.apply(lambda x : x.entity *100 / total, axis=1)
        dict_candidate[filename][col] = dict()
        dict_candidate[filename][col]['class_without_hr'] = list(df[['type','entity','vote_pct']].to_records(index=False))

In [ ]:
from tqdm import tqdm

for filename in tqdm(dict_cand):
    for col in dict_cand[filename]:
        dict_cand[filename][col]['class_with_hr'] = list([])
        for cls in dict_cand[filename][col]['class_without_hr']:
            if len([value for value in get_dbo_subclass(cls) if value in dict_cand[filename][col]['class_without_hr']]) == 0:
                dict_cand[filename][col]['class_with_hr'].append(cls)
                
with open(('output/dict_cand-%s.json' % time.strftime("%Y%m%d-%H%M%S")), 'w') as fp:
        json.dump(dict_cand, fp)

# Experiments:
 1. Top X without repetition of cell values, without checking parent child relationships
 2. Top x without repetition of cell values, having removed candidate classes for which at list one subclass also appears in the list of candidate classes
 3. Top x with repetition of cell values. This means if a value appears multiple times in a column then the cancidate class will get more that one votes from that cell value

In [11]:
def lookup_assessment(dict_cand, threshold = 10000):
    found = 0
    found_with = 0
    total_columns = 0

    for file in dict_cand:
        for col in dict_cand[file]:
            candidate_class_without_hr = dict_cand[file][col]['class_without_hr'][:threshold]
#             candidate_class_with_hr = dict_cand[file][col]['class_with_hr'][:threshold]
            actual_cls = data[file]['gt'][col]
            if actual_cls in candidate_class_without_hr:
                found+=1
#             if actual_cls in candidate_class_with_hr:
#                 found_with+=1
            total_columns+=1
    return (round(100*found/total_columns,2),round(100*found_with/total_columns,2))



def lookup_assessment_considering_hr(dict_cand, threshold = 10000):
    """
    This is a function that considers half a point (instead of a whole point) in case the predicted class is a parent of the expected type
    """
    found = 0
    found_with = 0
    parent_found = 0
    total_columns = 0

    for file in dict_cand:
        for col in dict_cand[file]:
            candidate_class_without_hr = dict_cand[file][col]['class_without_hr'][:threshold]
#             candidate_class_with_hr = dict_cand[file][col]['class_with_hr'][:threshold]
            actual_cls = data[file]['gt'][col]
            if actual_cls in candidate_class_without_hr:
                found+=1
                parent_found += 1
            # else we give half a point in case the predicted value is in the hierarchy (i.e. parent) of the actual value
            else:
                parents_of_expected_type = get_dbo_superclass(actual_cls)
                intersection = [value for value in parents_of_expected_type if value in candidate_class_without_hr]
                if len(intersection) > 0:
                    parent_found += 0.5
                    
#             if actual_cls in candidate_class_with_hr:
#                 found_with+=1
            total_columns+=1
    return (round(100*found/total_columns,2),round(100*found_with/total_columns,2),round(100*parent_found/total_columns,2))

In [12]:
dict_candidate['58891288_0_1117541047012405958']['1'].keys()

dict_keys(['class_without_hr'])

In [ ]:
results = []

# first we calculate the max number of candidates across all the columns in the tabular data so we cap the range
# max_cand = 0
# for file in dict_col_candidate_classes_rank2:
#     for col in dict_col_candidate_classes_rank2[file]:
        
#         candidate_cls = set()
#         for element in dict_col_candidate_classes_rank2[file][col]:
#             candidate_cls.add(element[0])
#         if len(candidate_cls) > max_cand:
#             max_cand = len(candidate_cls)

# the we increase the size of top list one element at a time until we reac the cap...
for i in range(1,16):
    (x,y) = (i, lookup_assessment(dict_candidate, i))
    results.append((x,y))

#... and plot the results
fig, ax = plt.subplots()
ax.plot([x[0] for x in results],[x[1] for x in results])
plt.show()
results

C:\Users\zacharias.detorakis\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:34: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
C:\Users\zacharias.detorakis\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:40: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.


In [ ]:
results = []

# first we calculate the max number of candidates across all the columns in the tabular data so we cap the range
max_cand = 0
for file in dict_col_candidate_classes_rank2:
    for col in dict_col_candidate_classes_rank2[file]:
        
        candidate_cls = set()
        for element in dict_col_candidate_classes_rank2[file][col]:
            candidate_cls.add(element[0])
        if len(candidate_cls) > max_cand:
            max_cand = len(candidate_cls)

# the we increase the size of top list one element at a time until we reac the cap...
for i in range(1,16):
    (x,y) = (i, lookup_assessment_considering_hr(dict_cand, i))
    results.append((x,y))

#... and plot the results
fig, ax = plt.subplots()
ax.plot([x[0] for x in results],[x[1] for x in results])
plt.show()
results

In [ ]:
results = []

# first we calculate the max number of candidates across all the columns in the tabular data so we cap the range
max_cand = 0
for file in dict_col_candidate_classes:
    for col in dict_col_candidate_classes[file]:
        
        candidate_cls = set()
        for element in dict_col_candidate_classes[file][col]:
            candidate_cls.add(element[0])
        if len(candidate_cls) > max_cand:
            max_cand = len(candidate_cls)

# the we increase the size of top list one element at a time until we reac the cap...
for i in range(1,10):
    (x,y) = (i, lookup_assessment(dict_cand, i))
    results.append((x,y))

#... and plot the results
fig, ax = plt.subplots()
ax.plot([x[0] for x in results],[x[1] for x in results])
plt.show()
results

Retrieve all folders under the cnn_models root

In [ ]:
# change the directory to where the models are
cnn_model_directory = os.getcwd()+'\\output\\cnn_models'

In [ ]:
def get_cnn_models(directory):
    temp = [x[0] for x in os.walk(directory)]
    temp.remove(directory)
    return set([x.replace(directory+'\\','').split('\\')[0] for x in temp])

trained_models = list(get_cnn_models(cnn_model_directory))
trained_models

In [ ]:
def load_model(cnn_model_directory, candidate_class):
    return keras.models.load_model(cnn_model_directory+'\%s' % candidate_class)

# model = load_model(cnn_model_directory, 'Astronaut')

In [ ]:
# model.summary()

In [ ]:
# results